In [1]:
# Translated to .py by Monique A. Stinson
# 31.12.2016
# Adapted to PandasBiogeme by Michel Bierlaire
# Tue Oct 23 16:12:40 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable, log

df = pd.read_csv("telephone.dat",'\t')
database = db.Database("telephone",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_BM	 = Beta('ASC_BM',0,None,None,0)
ASC_SM	 = Beta('ASC_SM',0,None,None,1)
ASC_LF	 = Beta('ASC_LF',0,None,None,0)
ASC_EF	 = Beta('ASC_EF',0,None,None,0)
ASC_MF	 = Beta('ASC_MF',0,None,None,0)
B_COST	 = Beta('B_COST',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

logcostBM  = DefineVariable('logcostBM', log(cost1),database)
logcostSM  = DefineVariable('logcostSM', log(cost2),database)
logcostLF  = DefineVariable('logcostLF', log(cost3),database)
logcostEF  = DefineVariable('logcostEF', log(cost4),database)
logcostMF  = DefineVariable('logcostMF', log(cost5),database)

#Utilities
V_BM = ASC_BM + B_COST * logcostBM
V_SM = ASC_SM + B_COST * logcostSM
V_LF = ASC_LF + B_COST * logcostLF
V_EF = ASC_EF + B_COST * logcostEF
V_MF = ASC_MF + B_COST * logcostMF

V = {1: V_BM, 2: V_SM, 3: V_LF, 4: V_EF, 5: V_MF}
avail = {1: avail1, 2: avail2, 3: avail3, 4: avail4, 5: avail5}

# The choice model is a logit, with availability conditions
logprob = models.loglogit(V,avail,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_Tel_generic"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


        Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_BM -0.721    0.154   -4.68  2.9e-06         0.152        -4.76   
ASC_EF  0.999    0.712     1.4    0.161         0.703         1.42   
ASC_LF    1.2    0.156    7.72 1.18e-14         0.159         7.56   
ASC_MF   1.74    0.276    6.28 3.31e-10         0.267         6.51   
B_COST  -2.03    0.214   -9.47        0         0.212        -9.55   

        Rob. p-value  
ASC_BM      1.94e-06  
ASC_EF         0.155  
ASC_LF      3.95e-14  
ASC_MF      7.28e-11  
B_COST             0  
Nbr of observations: 434
LL(0) =    -560.250
LL(beta) = -477.557
rho bar square = 0.139
Output file: logit_Tel_generic~00.html
